In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class Net(nn.Module):
    # Metodo costruttore / Schema della rete neurale
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        

        self.fc1 = nn.Linear(8* 56 * 56, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out



In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), # Trasformiamo l'immagine in un tensore
])

# Peschiamo i dati di training
dataset = datasets.ImageFolder(root='dataset', transform=transform)
print(dataset.class_to_idx)

# Suddividiamo il dataset in training e validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

{'cats': 0, 'dogs': 1}


In [6]:
model = Net()
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

model.train()
for epoch in range(15):
    running_loss = 0.0 # Rappresenta la loss del singolo batch
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(images) # Prima volta in cui passiamo le immagini nella rete
        loss = loss_fn(outputs, labels) # Calcoliamo la loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader) # Loss media su tutti i batch
    print(f"Epoch {epoch}, Loss: {avg_loss}")


Epoch 0, Loss: 0.6860524713993073
Epoch 1, Loss: 0.6737722635269165
Epoch 2, Loss: 0.647117656469345
Epoch 3, Loss: 0.6017773270606994
Epoch 4, Loss: 0.5721716940402984
Epoch 5, Loss: 0.5076314270496368
Epoch 6, Loss: 0.4637900412082672
Epoch 7, Loss: 0.41112600862979887
Epoch 8, Loss: 0.3859466940164566
Epoch 9, Loss: 0.2936780959367752
Epoch 10, Loss: 0.24542222470045089
Epoch 11, Loss: 0.20702120363712312
Epoch 12, Loss: 0.14328788593411446
Epoch 13, Loss: 0.13495455980300902
Epoch 14, Loss: 0.08506143242120742


In [5]:
image = Image.open('dataset\\cats\\cat (3).jpg').convert('RGB')
image_tensor = transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(image_tensor)
    probs = torch.softmax(output, dim=1)    
print(output)
print(f"{probs*100}%")

tensor([[ 0.9958, -4.2078]], device='cuda:0')
tensor([[99.4533,  0.5467]], device='cuda:0')%


In [9]:
model.load_state_dict(torch.load("C:\\Users\\rdarc\\Desktop\\Reti Neurali\\Codice\\image_classifier.pth", weights_only=True))


<All keys matched successfully>